In [6]:
#************************************************************************************
# Rezwan Matin
# Thesis B
# Filename: ML_MLP_RAVDESS_5.py
# Date: 2/23/20
#
# Objective:
# 26 MFCCs (mean) and 26 MFCCs (standard deviation), ZCR for BEST SGD so far!
#
#*************************************************************************************

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa as rosa
import glob
import os
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import tensorflow.keras as keras

# Save directory path in 'path'
path = r'C:/Books/Texas State Books/Fall 2019/Thesis A/Corpus/Simulated/RAVDESS/All'

# Declare a dummy Numpy array (row vector)
result_array = np.empty([1,54])

# Create a list of audio file names 'file_list'
file_list = os.listdir(path)

i=0

for filename in file_list:
    
    # Read WAV file. 'rosa.core.load' returns sampling frequency in 'fs' and audio signal in 'sig'
    sig, fs = rosa.core.load(path + '/' + file_list[i], sr=None)
    
    # Calculate the average mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    avg_mfcc_feat = np.mean(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26).T,axis=0)
    
    # Calculate the standard deviation of mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.std' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    std_mfcc_feat = np.std(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26).T,axis=0)
    
    # Calculate the average zero crossing rate (utterance-level feature) using 'rosa.feat.zero_crossing_rate()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    zcross_feat = rosa.feature.zero_crossing_rate(sig)
    avg_zcross_feat = np.mean(rosa.feature.zero_crossing_rate(y=sig).T,axis=0)
    
    # Append the three 1D arrays into a single 1D array called 'feat'.
    feat0 = np.append(avg_mfcc_feat, std_mfcc_feat, axis=0)
    
    feat1 = np.append(feat0, avg_zcross_feat, axis=0)
    
    # Save emotion label from file name. 'path' contains directory's address, 'file_list' contains file name, and '\\' joins the two to form file's address
    label = os.path.splitext(os.path.basename(path + '\\' + file_list[i]))[0].split('-')[2]
    
    # Create a new Numpy array 'sample' to store features along with label
    sample = np.insert(feat1, obj=53, values=label)
    
    result_array = np.append(result_array, sample)
    
    i+=1

# Print out the 1D Numpy array
result_array

result_array.shape

# Convert 1D Numpy array to 2D array. Argument must be a Tuple. i+1 because we have i samples (audio files) plus a dummy row.
result_array = np.reshape(result_array, (i+1,-1))

# Delete first dummy row from 2D array
result_array = np.delete(result_array, 0, 0)

# Print final 2D Numpy array 
result_array.shape

df = pd.DataFrame(data=result_array)
# Label only the last (target) column
df = df.rename({53: "Emotion"}, axis='columns')
# Delete unnecessary emotion data (calm)
df.drop(df[df['Emotion'] == 2.0].index, inplace = True)
# Reset row (sample) indexing
df = df.reset_index(drop=True)
df.tail(12)

df['Emotion'].value_counts()

# Balance the dataset for equal number of samples for each class.
# Separate majority and minority classes
df_minority = df[df.Emotion==1.0]
df_majority3 = df[df.Emotion==3.0]
df_majority4 = df[df.Emotion==4.0]
df_majority5 = df[df.Emotion==5.0]
df_majority6 = df[df.Emotion==6.0]
df_majority7 = df[df.Emotion==7.0]
df_majority8 = df[df.Emotion==8.0]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=192,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority3, df_majority4, df_majority5, df_majority6, df_majority7, df_majority8])
 
# Display new class counts
df_upsampled.Emotion.value_counts()

# Reset row (sample) indexing
df_upsampled = df_upsampled.reset_index(drop=True)

df_upsampled['Emotion'].value_counts()

# Extract target feature 'Emotion' in a vector y. Indexing from 0
y = df_upsampled.iloc[0:1344, 53].values
# Extract features 'buying' and 'safety' in a vector X. Indexing from 0
X = df_upsampled.iloc[0:1344, list(range(53))].values
print(y)

# Split data for training and testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

# Standardize the inputs
X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

np.random.seed(123)

tf.random.set_seed(123)

# One-Hot Encode the classes
y_train_onehot = keras.utils.to_categorical(y_train)

# Create an object/instance 'model' for the 'Sequential()' class.
model = keras.models.Sequential()
model.add(
    keras.layers.Dense( units=53,
                input_dim=X_train_centered.shape[1],
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros', 
                activation='selu'))
model.add(
    keras.layers.Dense( units=90,
                input_dim=53,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))
model.add(
    keras.layers.Dense( units=90,
                input_dim=90,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))
model.add(
    keras.layers.Dense( units=90,
                input_dim=90,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))

model.add(
    keras.layers.Dense( units=y_train_onehot.shape[1],
                input_dim=90,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='softmax'))

sgd_optimizer = keras.optimizers.SGD(
                    lr=0.01, decay=1e-7, momentum=0.9)

model.compile(optimizer=sgd_optimizer,
                    loss='categorical_crossentropy')
                          
                            # cross-entropy: fancy name for logistic regression
                            
# Train the MLP
history = model.fit(X_train_centered, y_train_onehot, batch_size=16, epochs=500, verbose=1, validation_split=0.1) # 90% training / 10% validation

y_train_pred = model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]

print('Training accuracy: %.2f%%' % (train_acc * 100))

y_test_pred = model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]

print('Test accuracy: %.2f%%' % (test_acc * 100))

[1. 1. 1. ... 8. 8. 8.]
(1008, 53) (1008,)
(336, 53) (336,)
Train on 907 samples, validate on 101 samples
Epoch 1/500
907/907 [==============================] - 1s 979us/sample - loss: 1.8343 - val_loss: 1.7226
Epoch 2/500
907/907 [==============================] - 0s 247us/sample - loss: 1.6364 - val_loss: 1.6136
Epoch 3/500
907/907 [==============================] - 0s 264us/sample - loss: 1.5524 - val_loss: 1.5791
Epoch 4/500
907/907 [==============================] - 0s 356us/sample - loss: 1.6564 - val_loss: 1.5161
Epoch 5/500
907/907 [==============================] - 0s 223us/sample - loss: 1.5257 - val_loss: 1.4547
Epoch 6/500
907/907 [==============================] - 0s 215us/sample - loss: 1.5100 - val_loss: 1.4528
Epoch 7/500
907/907 [==============================] - 0s 226us/sample - loss: 1.4362 - val_loss: 1.4177
Epoch 8/500
907/907 [==============================] - 0s 212us/sample - loss: 1.4800 - val_loss: 1.4656
Epoch 9/500
907/907 [==============================] -

Epoch 154/500
907/907 [==============================] - 0s 216us/sample - loss: 0.3821 - val_loss: 1.4293
Epoch 155/500
907/907 [==============================] - 0s 226us/sample - loss: 0.2751 - val_loss: 1.5020
Epoch 156/500
907/907 [==============================] - 0s 219us/sample - loss: 0.5756 - val_loss: 1.7930
Epoch 157/500
907/907 [==============================] - 0s 214us/sample - loss: 0.8033 - val_loss: 1.4149
Epoch 158/500
907/907 [==============================] - 0s 217us/sample - loss: 0.6395 - val_loss: 1.5467
Epoch 159/500
907/907 [==============================] - 0s 213us/sample - loss: 0.5835 - val_loss: 1.3667
Epoch 160/500
907/907 [==============================] - 0s 226us/sample - loss: 0.4278 - val_loss: 1.3935
Epoch 161/500
907/907 [==============================] - 0s 219us/sample - loss: 0.4433 - val_loss: 1.6083
Epoch 162/500
907/907 [==============================] - 0s 217us/sample - loss: 0.4732 - val_loss: 1.6892
Epoch 163/500
907/907 [==============

907/907 [==============================] - 0s 215us/sample - loss: 0.2627 - val_loss: 2.1504
Epoch 307/500
907/907 [==============================] - 0s 215us/sample - loss: 0.2264 - val_loss: 2.7836
Epoch 308/500
907/907 [==============================] - 0s 216us/sample - loss: 0.6443 - val_loss: 3.5813
Epoch 309/500
907/907 [==============================] - 0s 225us/sample - loss: 0.4771 - val_loss: 2.0542
Epoch 310/500
907/907 [==============================] - 0s 218us/sample - loss: 0.2179 - val_loss: 1.9051
Epoch 311/500
907/907 [==============================] - 0s 215us/sample - loss: 0.2757 - val_loss: 2.1935
Epoch 312/500
907/907 [==============================] - 0s 217us/sample - loss: 0.1247 - val_loss: 2.1206
Epoch 313/500
907/907 [==============================] - 0s 217us/sample - loss: 0.1732 - val_loss: 3.3787
Epoch 314/500
907/907 [==============================] - 0s 219us/sample - loss: 0.3244 - val_loss: 2.5707
Epoch 315/500
907/907 [============================

Epoch 381/500
907/907 [==============================] - 0s 218us/sample - loss: 1.3346e-04 - val_loss: 1.8220
Epoch 382/500
907/907 [==============================] - 0s 212us/sample - loss: 1.3399e-04 - val_loss: 1.8235
Epoch 383/500
907/907 [==============================] - 0s 209us/sample - loss: 1.3103e-04 - val_loss: 1.8242
Epoch 384/500
907/907 [==============================] - 0s 212us/sample - loss: 1.2987e-04 - val_loss: 1.8246
Epoch 385/500
907/907 [==============================] - 0s 344us/sample - loss: 1.2694e-04 - val_loss: 1.8255
Epoch 386/500
907/907 [==============================] - 1s 717us/sample - loss: 1.2656e-04 - val_loss: 1.8275
Epoch 387/500
907/907 [==============================] - 0s 209us/sample - loss: 1.2569e-04 - val_loss: 1.8292
Epoch 388/500
907/907 [==============================] - 0s 206us/sample - loss: 1.2382e-04 - val_loss: 1.8285
Epoch 389/500
907/907 [==============================] - 0s 211us/sample - loss: 1.2358e-04 - val_loss: 1.8299
E

Epoch 455/500
907/907 [==============================] - 0s 215us/sample - loss: 7.1840e-05 - val_loss: 1.8676
Epoch 456/500
907/907 [==============================] - 0s 215us/sample - loss: 7.1583e-05 - val_loss: 1.8678
Epoch 457/500
907/907 [==============================] - 0s 218us/sample - loss: 7.0878e-05 - val_loss: 1.8688
Epoch 458/500
907/907 [==============================] - 0s 216us/sample - loss: 7.0836e-05 - val_loss: 1.8691
Epoch 459/500
907/907 [==============================] - 0s 214us/sample - loss: 7.0156e-05 - val_loss: 1.8684
Epoch 460/500
907/907 [==============================] - 0s 218us/sample - loss: 6.9764e-05 - val_loss: 1.8699
Epoch 461/500
907/907 [==============================] - 0s 224us/sample - loss: 6.9669e-05 - val_loss: 1.8703
Epoch 462/500
907/907 [==============================] - 0s 215us/sample - loss: 6.8838e-05 - val_loss: 1.8703
Epoch 463/500
907/907 [==============================] - 0s 216us/sample - loss: 6.8820e-05 - val_loss: 1.8710
E